# Notes

GraphPCA

In [1]:
import GraphPCA as sg
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import squidpy as sq
import scipy
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances as pair
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.neighbors import kneighbors_graph

# Load data

These two files could be donwloaded from [IO files for reproducing](../../README.md#other-links)

In [2]:
raw_counts = pd.read_csv('../../../STdata/stereo_mouse_mid_brain/stereo_seq_raw_counts.csv', index_col=0)
raw_counts.head()

,E12.5_E1S3_99895,E12.5_E1S3_99913,E12.5_E1S3_99944,E12.5_E1S3_100014,E12.5_E1S3_100033,E12.5_E1S3_100034,E12.5_E1S3_100035,E12.5_E1S3_100047,E12.5_E1S3_100081,E12.5_E1S3_100100,...,E16.5_E2S7_326356,E16.5_E2S7_326357,E16.5_E2S7_326359,E16.5_E2S7_326384,E16.5_E2S7_326390,E16.5_E2S7_326391,E16.5_E2S7_326412,E16.5_E2S7_326433,E16.5_E2S7_326448,E16.5_E2S7_326458
0610005C13Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610006L08Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009B22Rik,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
0610009O20Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610010F05Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
raw_counts.columns = [x[:3]+x[5:] for x in raw_counts.columns]
raw_counts.head()

,E12_E1S3_99895,E12_E1S3_99913,E12_E1S3_99944,E12_E1S3_100014,E12_E1S3_100033,E12_E1S3_100034,E12_E1S3_100035,E12_E1S3_100047,E12_E1S3_100081,E12_E1S3_100100,...,E16_E2S7_326356,E16_E2S7_326357,E16_E2S7_326359,E16_E2S7_326384,E16_E2S7_326390,E16_E2S7_326391,E16_E2S7_326412,E16_E2S7_326433,E16_E2S7_326448,E16_E2S7_326458
0610005C13Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610006L08Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009B22Rik,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
0610009O20Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610010F05Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# # no genes with zero express
# sum(raw_counts.values.sum(axis=1)==0)

In [5]:
# np.isnan(raw_counts.values).sum()

In [6]:
# print(raw_counts.shape)

In [2]:
ontrac_input = pd.read_csv('../../../STdata/stereo_mouse_mid_brain/stereo_input.csv')
print(ontrac_input.shape)
ontrac_input.head()

(26621, 5)


,Cell_ID,Sample,Cell_Type,x,y
0,E12_E1S3_100034,E12_E1S3,Fibro,15940.0,18584.0
1,E12_E1S3_100035,E12_E1S3,Fibro,15942.0,18623.0
2,E12_E1S3_100191,E12_E1S3,Endo,15965.0,18619.0
3,E12_E1S3_100256,E12_E1S3,Fibro,15969.0,18717.0
4,E12_E1S3_100264,E12_E1S3,Fibro,15974.0,18692.0


# run GraphPCA

note: The results is not stable in different runs

In [ ]:
samples = ontrac_input['Sample'].unique().tolist()

In [ ]:
Batch_list = []
adj_list = []


for sample in samples:
    print(sample)
    
    # extract
    cells = ontrac_input[ontrac_input['Sample'] == sample]['Cell_ID']
    sample_counts = raw_counts[cells]
    
    # create adata
    adata = ad.AnnData(sample_counts.T)
    adata.obs = adata.obs.join(ontrac_input[['Cell_Type','Sample']])
    location = ontrac_input[ontrac_input['Sample'] == sample][['x','y']].values
    adata.uns["spatial"] = location
    
    # Constructing the spatial network
    n_neighbors = 7
    graph = kneighbors_graph(np.asarray(location), int(n_neighbors), metric='euclidean',
                                     metric_params={}, include_self=False)
    graph = 0.5 * (graph + graph.T)

    adata.uns["adj"] = graph
    adata.obsm["spatial"] = location
    
    # Normalization
    sc.pp.filter_genes(adata, min_cells=20)
    sc.experimental.pp.normalize_pearson_residuals(adata)
    sc.pp.scale(adata)

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)
    
    print(adata.X.shape)
    print(graph.shape)

In [ ]:
adata_concat = ad.concat(Batch_list, label="Sample", keys=samples)
print('adata_concat.shape: ', adata_concat.shape)

In [ ]:
# Normalization
sc.experimental.pp.normalize_pearson_residuals(adata_concat)
sc.pp.scale(adata_concat)

In [ ]:
%%time

from functools import reduce

adj_concat = reduce(scipy.linalg.block_diag, (x.todense() for x in adj_list))
adata_concat.uns['edgeList'] = np.nonzero(adj_concat)
adata_concat.uns['network'] = scipy.sparse.csr_matrix(adj_concat)

In [ ]:
%%time

Z,_ = sg.Run_GPCA(adata_concat, network=adata_concat.uns['network'], n_components = 50, method = "knn", _lambda = 0.6,n_neighbors=7,
               save_reconstruction=True)
adata_concat.obsm["GraphPCA"] = Z
print(Z.shape)

In [ ]:
estimator = KMeans(n_clusters=6)
res = estimator.fit(Z[:,:])
lable_pred=res.labels_
adata_concat.obs["GPCA_pred"]= lable_pred
adata_concat.obs["GPCA_pred"] = adata_concat.obs["GPCA_pred"].astype('category')
adata_concat.obsm["GraphPCA"] = Z


In [ ]:
adata_concat.obs = adata_concat.obs.join(ontrac_input.set_index('Cell_ID')[['Cell_Type', 'x','y']])
adata_concat.obs.head()

In [ ]:
adata_concat.obs.to_csv('GPCA_pred.csv', index=True, index_label='Cell_ID')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = 'Arial'
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
%matplotlib inline

In [ ]:
sample_df = adata_concat.obs[adata_concat.obs['Sample'] == 'E14_E1S3']
sample_df['GPCA_pred'] = sample_df['GPCA_pred'].astype('category')

with sns.axes_style('white', rc={
        'xtick.bottom': True,
        'ytick.left': True
}), sns.plotting_context('paper',
                         rc={
                             'axes.titlesize': 8,
                             'axes.labelsize': 8,
                             'xtick.labelsize': 6,
                             'ytick.labelsize': 6,
                             'legend.fontsize': 6
                         }):
    fig, ax = plt.subplots()
    sns.scatterplot(data = sample_df, x='x',y='y', hue=sample_df['GPCA_pred'], ax=ax)

# variations with in domain

In [ ]:
meta_df = pd.read_csv('GPCA_pred.csv', index_col=0)

In [8]:
NT_score = pd.read_csv('../../output/stereo_midbrain_base_NT/NTScore.csv.gz', index_col=0)
NT_score.head()

,x,y,Niche_NTScore,Cell_NTScore
Cell_ID,,,,
E12_E1S3_100034,15940.0,18584.0,0.113668,0.198802
E12_E1S3_100035,15942.0,18623.0,0.091417,0.132273
E12_E1S3_100191,15965.0,18619.0,0.092212,0.161531
E12_E1S3_100256,15969.0,18717.0,0.090893,0.091927
E12_E1S3_100264,15974.0,18692.0,0.090893,0.100961


In [ ]:
meta_filtered_df = meta_df[meta_df['Sample'] == 'E14_E1S3']
meta_filtered_df = meta_filtered_df[meta_filtered_df['Cell_Type']=='RGC']
print(meta_filtered_df.shape)
meta_filtered_df.head()

In [ ]:
meta_filtered_3_df = meta_filtered_df[meta_filtered_df['GPCA_pred'] == '3']
print(meta_filtered_3_df.shape)

(362, 8)
(80, 8)


In [ ]:
meta_filtered_3_df = meta_filtered_3_df.join(NT_score['Cell_NTScore'])

In [13]:
scaled_exp = pd.read_csv('/sc/arion/projects/YuanLab/gcproj/wen/ONTrac_test/analysis/stereo_seq/stereo_seq_scaled_exp.txt', index_col=0, sep='\t')
scaled_exp.head()

,E12_E1S3_100034,E12_E1S3_100035,E12_E1S3_100191,E12_E1S3_100256,E12_E1S3_100264,E12_E1S3_100337,E12_E1S3_100346,E12_E1S3_100461,E12_E1S3_100502,E12_E1S3_100532,...,E16_E2S7_326320,E16_E2S7_326323,E16_E2S7_326324,E16_E2S7_326325,E16_E2S7_326329,E16_E2S7_326357,E16_E2S7_326359,E16_E2S7_326384,E16_E2S7_326391,E16_E2S7_326412
0610009B22Rik,-0.164753,-0.145323,-0.169342,-0.146486,-0.141857,-0.182881,-0.180363,-0.174450,-0.199164,-0.164989,...,-0.186833,-0.180521,-0.140679,-0.156495,-0.172933,-0.110040,-0.138963,-0.144868,-0.174496,-0.127659
0610010F05Rik,-0.202786,-0.179762,-0.208226,-0.181139,-0.175657,-0.224283,-0.221297,-0.214282,-0.243610,-0.203065,...,-0.228973,-0.221484,-0.174262,-0.192998,-0.212484,-0.156872,-0.172230,-0.179224,-0.214338,-0.165153
0610010K14Rik,-0.280477,-0.251139,-0.287405,-0.252895,5.874698,-0.307839,-0.304040,-0.295114,-0.332409,-0.280833,...,-0.313804,-0.304278,-0.244126,-0.268010,-0.292825,3.544289,-0.241534,-0.250453,-0.295184,-0.232507
0610012G03Rik,1.874134,-0.252851,-0.290147,-0.254657,-0.247466,-0.311154,-0.307249,-0.298073,0.849398,-0.283390,...,-0.317286,-0.307494,-0.245637,-0.270202,-0.295720,-0.222812,-0.242970,-0.252145,-0.298145,-0.233683
0610037L13Rik,-0.174357,-0.154231,-0.179111,-0.155435,-0.150641,-0.193134,-0.190527,-0.184401,-0.210000,-0.174602,...,-0.197228,-0.190690,-0.149421,-0.165804,-0.182831,-0.121942,-0.147643,-0.153760,-0.184449,-0.140846


## within cluster variation

In [ ]:
meta_filtered_3_df.head()

In [15]:
meta_filtered_3_df['Cell_NTScore_r'] = 1-meta_filtered_3_df['Cell_NTScore']

In [17]:
cells_low = meta_filtered_3_df[meta_filtered_3_df['Cell_NTScore_r']<0.17].index.tolist()
cells_high = meta_filtered_3_df[meta_filtered_3_df['Cell_NTScore_r']>=0.17].index.tolist()

In [18]:
genes = [
    'Cdk8','Pum3','Nkd1','Hacd3','Tet3','Efna5','Cd24a','Ccnd2','Rpl10','Rrm2','Rpa2','Mrps18c','Phb2'
]


In [19]:
data_df = scaled_exp.T.loc[[*cells_low,*cells_high],genes]
data_df['group'] = ['NT-Low RGC'] * len(cells_low) + ['NT-High RGC'] * len(cells_high)

In [23]:
print(len(cells_low))
print(len(cells_high))

21
59


In [20]:
data_df.head()

,Cdk8,Pum3,Nkd1,Hacd3,Tet3,Efna5,Cd24a,Ccnd2,Rpl10,Rrm2,Rpa2,Mrps18c,Phb2,group
E14_E1S3_177312,8.013305,-0.133016,-0.183434,9.989462,-0.168696,-0.349939,-0.622267,-0.338743,-0.658535,-0.172963,-0.131967,-0.222288,-0.254253,NT-Low RGC
E14_E1S3_177389,3.087999,-0.132633,-0.182944,-0.194911,-0.168218,-0.349134,-0.620698,-0.337947,6.018661,-0.172508,-0.131206,-0.221686,-0.253598,NT-Low RGC
E14_E1S3_177402,2.008103,-0.145273,-0.199111,-0.212182,-0.183979,-0.375693,-0.672682,-0.364224,0.199452,-0.187527,-0.143834,-0.241542,-0.275226,NT-Low RGC
E14_E1S3_177425,-0.245940,-0.181350,-0.245231,-0.261485,-0.228924,-0.451276,-0.823176,-0.438998,0.026098,-0.230340,-0.178785,-0.298192,4.196295,NT-Low RGC
E14_E1S3_177448,2.243143,-0.155650,-0.212382,-0.226363,-0.196914,9.002382,-0.715633,-0.385770,0.005892,-0.199851,-0.153884,-0.257841,-0.292968,NT-Low RGC


In [22]:
from scipy.stats import ttest_ind, mannwhitneyu

for g in genes:
    ttest_s, ttest_p, *_ = ttest_ind(data_df[g].values[:len(cells_low)],data_df[g].values[len(cells_low):])
    mann_w_u_s, mann_w_u_p =  mannwhitneyu(data_df[g].values[:len(cells_low)],data_df[g].values[len(cells_low):])
    print(g, ttest_s, ttest_p, mann_w_u_s, mann_w_u_p)

Cdk8 2.658919341983387 0.009508915745094128 821.0 0.027956020517882478
Pum3 -1.6450366312129443 0.10398664923713649 447.0 0.05999713741278215
Nkd1 -1.3752804186074472 0.17298170475590513 470.0 0.10325137471016226
Hacd3 0.3307587464867543 0.7417135941403635 505.0 0.2125518526618987
Tet3 -1.1348062443451326 0.25993182587316105 511.0 0.23761750352290179
Efna5 3.4413788063126813 0.0009329400191045064 827.0 0.023603327500390747
Cd24a -0.5367400761083073 0.5929748642352881 504.0 0.20856755488237333
Ccnd2 -0.40664450766200544 0.6853827442070554 598.0 0.8183769614717933
Rpl10 2.792068854394023 0.006583812551436758 889.0 0.003266380372977806
Rrm2 -0.7890719564400255 0.43246190954676633 485.0 0.142845293479777
Rpa2 -0.6017023252044652 0.5491172445592444 532.0 0.34143064519334954
Mrps18c 1.8887109565749363 0.06264797638269641 578.0 0.6539142618651163
Phb2 2.0365854299491146 0.045085103376720295 617.0 0.9825518074978354
